In [1]:
import sqlite3
import pandas as pd

In [5]:
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

In [6]:
sql_script = r"""
CREATE TABLE productos (
  producto_id INTEGER PRIMARY KEY,
  sku TEXT NOT NULL UNIQUE,
  nombre TEXT NOT NULL,
  costo_unitario DECIMAL(10,2) NOT NULL
);
CREATE TABLE almacenes (
  almacen_id INTEGER PRIMARY KEY,
  nombre TEXT NOT NULL,
  ciudad TEXT
);
CREATE TABLE inventario_actual (
  inventario_id INTEGER PRIMARY KEY,
  producto_id INTEGER NOT NULL REFERENCES productos(producto_id),
  almacen_id INTEGER NOT NULL REFERENCES almacenes(almacen_id),
  stock_actual INTEGER NOT NULL,
  UNIQUE(producto_id, almacen_id)
);
CREATE TABLE movimientos_almacen (
  movimiento_id INTEGER PRIMARY KEY,
  fecha DATE NOT NULL,
  producto_id INTEGER NOT NULL REFERENCES productos(producto_id),
  almacen_id INTEGER NOT NULL REFERENCES almacenes(almacen_id),
  tipo TEXT NOT NULL CHECK (tipo IN ('ENTRADA','SALIDA')),
  cantidad INTEGER NOT NULL CHECK (cantidad > 0)
);
CREATE TABLE clientes (
  cliente_id INTEGER PRIMARY KEY,
  nombre TEXT NOT NULL,
  ciudad TEXT
);
CREATE TABLE ordenes (
  orden_id INTEGER PRIMARY KEY,
  cliente_id INTEGER NOT NULL REFERENCES clientes(cliente_id),
  producto_id INTEGER NOT NULL REFERENCES productos(producto_id),
  cantidad_ordenada INTEGER NOT NULL CHECK (cantidad_ordenada > 0),
  fecha_orden DATE NOT NULL,
  fecha_promesa DATE NOT NULL
);
CREATE TABLE entregas (
  entrega_id INTEGER PRIMARY KEY,
  orden_id INTEGER NOT NULL UNIQUE REFERENCES ordenes(orden_id),
  fecha_entrega DATE,
  cantidad_entregada INTEGER NOT NULL CHECK (cantidad_entregada >= 0),
  transportista TEXT,
  estado TEXT NOT NULL CHECK (estado IN ('ENTREGADA','PENDIENTE','CANCELADA'))
);
INSERT INTO productos VALUES
  (1, 'A-001', 'Aceite motor 1L', 95.00),
  (2, 'B-010', 'Filtro aire', 120.00),
  (3, 'C-200', 'Caja cartón M', 12.50),
  (4, 'D-310', 'Tarima madera', 180.00),
  (5, 'E-515', 'Anticongelante 5L', 260.00);
INSERT INTO almacenes VALUES
  (1, 'CDMX Centro', 'CDMX'),
  (2, 'Guadalajara Occidente', 'Guadalajara');
INSERT INTO inventario_actual VALUES
  (1, 1, 1, 180),
  (2, 1, 2,  90),
  (3, 2, 1,  50),
  (4, 2, 2,  30),
  (5, 3, 1, 500),
  (6, 3, 2, 350),
  (7, 4, 1,  25),
  (8, 4, 2,  15),
  (9, 5, 1,  40),
  (10,5, 2,  20);
INSERT INTO movimientos_almacen VALUES
  (1, '2025-09-01', 1, 1, 'ENTRADA', 200),
  (2, '2025-09-01', 3, 1, 'ENTRADA', 600),
  (3, '2025-09-03', 2, 2, 'ENTRADA', 100),
  (4, '2025-09-05', 5, 1, 'ENTRADA', 80),
  (5, '2025-09-07', 1, 1, 'SALIDA', 60),
  (6, '2025-09-08', 3, 1, 'SALIDA', 250),
  (7, '2025-09-09', 2, 2, 'SALIDA', 70),
  (8, '2025-09-10', 5, 1, 'SALIDA', 30),
  (9, '2025-09-11', 4, 2, 'SALIDA', 10),
  (10,'2025-09-12', 4, 1, 'SALIDA', 12);
INSERT INTO clientes VALUES
  (1, 'Refaccionaria Norte', 'CDMX'),
  (2, 'Autos Jalisco', 'Guadalajara'),
  (3, 'Logitec Distribución', 'CDMX');
INSERT INTO ordenes VALUES
  (1001, 1, 1, 40, '2025-09-08', '2025-09-10'),
  (1002, 1, 3, 80, '2025-09-08', '2025-09-11'),
  (1003, 2, 2, 35, '2025-09-09', '2025-09-12'),
  (1004, 2, 5, 20, '2025-09-10', '2025-09-13'),
  (1005, 3, 4, 12, '2025-09-10', '2025-09-15'),
  (1006, 3, 1, 60, '2025-09-11', '2025-09-15'),
  (1007, 1, 5, 15, '2025-09-12', '2025-09-14'),
  (1008, 2, 3, 90, '2025-09-12', '2025-09-16');
INSERT INTO entregas VALUES
  (1, 1001, '2025-09-10', 40, 'Paquexpress', 'ENTREGADA'),
  (2, 1002, '2025-09-12', 80, 'Estafeta', 'ENTREGADA'),
  (3, 1003, '2025-09-12', 30, 'DHL', 'ENTREGADA'),
  (4, 1004, '2025-09-13', 20, 'DHL', 'ENTREGADA'),
  (5, 1005, '2025-09-16', 12, '3PL Local', 'ENTREGADA'),
  (6, 1006, NULL,  0,  'Paquexpress', 'PENDIENTE'),
  (7, 1007, '2025-09-14', 15, 'Estafeta', 'ENTREGADA'),
  (8, 1008, '2025-09-15', 85, 'DHL', 'ENTREGADA');
"""
cur.executescript(sql_script)
conn.commit()

In [8]:
consultas = {
    "On Time Delivery Rate": """
        WITH entregas_filtradas AS (
            SELECT e.fecha_entrega, o.fecha_promesa
            FROM entregas e
            JOIN ordenes o ON o.orden_id = e.orden_id
            WHERE e.estado = 'ENTREGADA'
        )
        SELECT ROUND(AVG(CASE WHEN fecha_entrega <= fecha_promesa THEN 1.0 ELSE 0.0 END), 2) AS on_time_rate
        FROM entregas_filtradas;
    """,

    "In Full Rate": """
        WITH entregas_vs_orden AS (
            SELECT e.cantidad_entregada, o.cantidad_ordenada
            FROM entregas e
            JOIN ordenes o ON o.orden_id = e.orden_id
            WHERE e.estado = 'ENTREGADA'
        )
        SELECT ROUND(AVG(CASE WHEN cantidad_entregada >= cantidad_ordenada THEN 1.0 ELSE 0.0 END), 2) AS in_full_rate
        FROM entregas_vs_orden;
    """,

    "OTIF": """
        WITH base AS (
            SELECT e.fecha_entrega, e.cantidad_entregada, o.fecha_promesa, o.cantidad_ordenada
            FROM entregas e
            JOIN ordenes o ON o.orden_id = e.orden_id
            WHERE e.estado = 'ENTREGADA'
        )
        SELECT ROUND(AVG(
            CASE WHEN fecha_entrega <= fecha_promesa AND cantidad_entregada >= cantidad_ordenada
                 THEN 1.0 ELSE 0.0 END
        ), 2) AS otif_rate
        FROM base;
    """,

    "Lead Time Promedio (días)": """
        SELECT ROUND(AVG(julianday(e.fecha_entrega) - julianday(o.fecha_orden)), 2) AS lead_time_promedio_dias
        FROM entregas e
        JOIN ordenes o ON o.orden_id = e.orden_id
        WHERE e.estado = 'ENTREGADA' AND e.fecha_entrega IS NOT NULL;
    """,

    "Rotación de productos": """
        SELECT p.sku, p.nombre,
               SUM(CASE WHEN m.tipo='SALIDA' THEN m.cantidad ELSE 0 END) AS unidades_vendidas
        FROM movimientos_almacen m
        JOIN productos p ON p.producto_id = m.producto_id
        GROUP BY p.sku, p.nombre
        ORDER BY unidades_vendidas DESC;
    """,

    "Inventario valorizado": """
        SELECT a.nombre AS almacen,
               ROUND(SUM(ia.stock_actual * p.costo_unitario), 2) AS valor_inventario
        FROM inventario_actual ia
        JOIN productos p ON p.producto_id = ia.producto_id
        JOIN almacenes a ON a.almacen_id = ia.almacen_id
        GROUP BY a.nombre
        ORDER BY valor_inventario DESC;
    """,

    "Backorders": """
        SELECT o.orden_id, c.nombre AS cliente, p.nombre AS producto,
               o.cantidad_ordenada, e.cantidad_entregada, e.estado,
               o.fecha_promesa, e.fecha_entrega
        FROM ordenes o
        LEFT JOIN entregas e ON e.orden_id = o.orden_id
        JOIN clientes c ON c.cliente_id = o.cliente_id
        JOIN productos p ON p.producto_id = o.producto_id
        WHERE e.estado != 'ENTREGADA' OR e.cantidad_entregada < o.cantidad_ordenada
        ORDER BY o.fecha_promesa;
    """,

    "Alertas de inventario bajo": """
        SELECT p.sku, p.nombre, a.nombre AS almacen, ia.stock_actual
        FROM inventario_actual ia
        JOIN productos p ON p.producto_id = ia.producto_id
        JOIN almacenes a ON a.almacen_id = ia.almacen_id
        WHERE ia.stock_actual < 30
        ORDER BY ia.stock_actual ASC;
    """,

    "Puntualidad por transportista": """
        WITH base AS (
            SELECT e.transportista, e.fecha_entrega, o.fecha_promesa
            FROM entregas e
            JOIN ordenes o ON o.orden_id = e.orden_id
            WHERE e.estado='ENTREGADA'
        )
        SELECT transportista,
               ROUND(AVG(CASE WHEN fecha_entrega <= fecha_promesa THEN 1.0 ELSE 0.0 END), 2) AS on_time_rate
        FROM base
        GROUP BY transportista
        ORDER BY on_time_rate DESC;
    """,

    "Retraso promedio por transportista": """
        WITH base AS (
            SELECT (julianday(e.fecha_entrega) - julianday(o.fecha_promesa)) AS delta_dias, e.transportista
            FROM entregas e
            JOIN ordenes o ON o.orden_id = e.orden_id
            WHERE e.estado='ENTREGADA' AND e.fecha_entrega IS NOT NULL
        )
        SELECT transportista, ROUND(AVG(delta_dias), 2) AS retraso_promedio_dias
        FROM base
        GROUP BY transportista
        ORDER BY retraso_promedio_dias ASC;
    """,

    "Fill Rate por producto": """
        WITH base AS (
          SELECT o.producto_id,
                 1.0 * e.cantidad_entregada / NULLIF(o.cantidad_ordenada,0) AS fill_rate
          FROM ordenes o
          JOIN entregas e ON e.orden_id = o.orden_id
          WHERE e.estado='ENTREGADA'
        )
        SELECT p.sku, p.nombre, ROUND(AVG(fill_rate), 2) AS fill_rate_promedio
        FROM base
        JOIN productos p ON p.producto_id = base.producto_id
        GROUP BY p.sku, p.nombre
        ORDER BY fill_rate_promedio DESC;
    """
}

In [9]:
for nombre, consulta in consultas.items():
    print(f"\n {nombre}")
    display(pd.read_sql_query(consulta, conn))

print("\n Base de datos logística creada y KPIs calculados correctamente.")


 On Time Delivery Rate


,on_time_rate
0,0.71



 In Full Rate


,in_full_rate
0,0.71



 OTIF


...


 Retraso promedio por transportista


,transportista,retraso_promedio_dias
0,DHL,-0.33
1,Paquexpress,0.00
2,Estafeta,0.50
3,3PL Local,1.00



 Fill Rate por producto


,sku,nombre,fill_rate_promedio
0,A-001,Aceite motor 1L,1.00
1,D-310,Tarima madera,1.00
2,E-515,Anticongelante 5L,1.00
3,C-200,Caja cartón M,0.97
4,B-010,Filtro aire,0.86



 Base de datos logística creada y KPIs calculados correctamente.
